In [35]:
%matplotlib widget

In [36]:
import http.client
import requests
import datetime
import pandas as pd

In [37]:
import matplotlib.pyplot as plt
#plt.rcParams['figure.figsize'] = (10.0, 10.0)
#import seaborn as sns
#sns.set_context('talk')
#sns.set_style("darkgrid")


In [38]:
def ms_timestamp(dt):
    return int(dt.timestamp()*1000)

In [39]:
with open('emoncms.key') as f: key=f.read()

end = datetime.datetime.now()
start = end - datetime.timedelta(days=3)

reqmap = {
    'apikey' : key,
    'id' : 2,
    'start' : ms_timestamp(start),
    'end' : ms_timestamp(end),
    'interval' : 60
}

url = 'http://emonpi/feed/data.json'
response = requests.get(url, reqmap)

print(response.status_code)
print(response.headers)
print(response.text[:100])



200
{'Date': 'Wed, 31 Mar 2021 17:01:10 GMT', 'Server': 'Apache/2.4.38 (Raspbian)', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'Content-Length': '24822', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Content-Type': 'application/json'}
[[1616950870000,177],[1616950930000,177],[1616950990000,177],[1616951050000,177],[1616951110000,177]


In [40]:
df=pd.DataFrame([(datetime.datetime.fromtimestamp(t/1000), d) for t,d in response.json()], columns=['Time', 'Wh'])
df=df.set_index('Time')
df

,Wh
Time,
2021-03-28 18:01:10,177.0
2021-03-28 18:02:10,177.0
2021-03-28 18:03:10,177.0
2021-03-28 18:04:10,177.0
2021-03-28 18:05:10,177.0
...,...
2021-03-31 18:00:47,90128.0
2021-03-31 18:00:52,90131.0
2021-03-31 18:00:58,90135.0


In [41]:
df.index[1] - df.index[0]

Timedelta('0 days 00:01:00')

In [42]:
ax=df.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
minute_bars = df.resample('1min').last().diff() # Wh per 1 min bar
minute_bars = minute_bars * 60 # multiply by 60 to get Watts for each bar
minute_bars = minute_bars / 1000 # divide by 1000 to convert to kWatts
minute_bars = minute_bars.rename(columns={'Wh' : 'kW'})
ax=minute_bars.plot()
ax.set_ylabel('kWatts');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
ax=minute_bars.plot.hist(bins=75)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
#Validate against meter readings
readings = [
    (pd.to_datetime("2021-03-31 10:46:00"), 122313),
    (pd.to_datetime("2021-03-31 17:39:00"), 122321)]
wh1 = df.iloc[df.index.get_loc(readings[0][0], method='nearest')].Wh
wh2 = df.iloc[df.index.get_loc(readings[1][0], method='nearest')].Wh

print(f"Emoncms thinks we used {(wh2-wh1)/1000}kWh")
print(f"Meter   thinks we used {(readings[1][1]-readings[0][1])}kWh")


Emoncms thinks we used 7.1kWh
Meter   thinks we used 8kWh
